Mitq tutorial: https://mitiq.readthedocs.io/en/stable/examples/ibmq-backends.html

# Imports

In [1]:
import sys
import os
# Add parent directory to path to import utils
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise

from utils.pce_vs_zne_utils import (ibmq_executor_pcs, mitigate_zne, extrapolate_checks, 
                                          load_or_generate_random_cliffs, get_ideal_expectation, save_avg_errors)

from utils.pauli_checks import convert_to_PCS_circ

from qiskit import *
from mitiq import zne
from qiskit_ibm_runtime.fake_provider import *

np.set_printoptions(precision=6, edgeitems=10, linewidth=150, suppress=True)

In [2]:
import os
import re
from typing import Tuple, Optional

def data_exists(
    folder: str,
    num_qubits: int,
    depth: int,
    num_circs: int,
    num_samp: int
) -> Tuple[bool, Optional[str]]:
    """
    Check if a CSV with matching parameters already exists in `folder`,
    ignoring any '_cx=...' part of the filename.

    Returns
    -------
    exists : bool
      True if at least one matching file is found.
    path   : str or None
      The path of the first matching file, or None.
    """
    # Allow any characters (including dots) in the _cx= value
    pattern = re.compile(
        rf"^avg_errors_n={num_qubits}_d={depth}"
        rf"_num_circs={num_circs}_num_samp={num_samp}"
        r"(?:_cx=[^/]+)?\.csv$"
    )
    for fn in os.listdir(folder):
        if pattern.match(fn):
            return True, os.path.join(folder, fn)
    return False, None


#### Backend settings

In [3]:
USE_REAL_HARDWARE = True

In [4]:
# Custom nosie model
prob_1 = 5e-4 # 0.0005
prob_2 = 5e-3 # 0.005

error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'sx', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

In [5]:
# service = QiskitRuntimeService()
# print(service.instances())

In [ ]:
backend = AerSimulator(noise_model=noise_model, method="stabilizer")

print(backend)

# Run Experiments 

In [ ]:
circuit_folder = "data_PCE_vs_ZNE/saved_circs"

# overwrite = False will skip experiments if data exists; True will re-run everything
overwrite = False

# Set parameters to sweep over
qubit_list = [6, 8, 10, 12]
depth_list  = range(20, 61, 10)
num_circs   = 20
# shot_budgets = range(10_000, 100_001, 20_000)
shot_budgets = [100_000]
for shot in shot_budgets:
    print(shot)

# ZNE parameters 
zne_methods_list   = ["linear", "richardson"]
scale_factors_list  = [
    [1, 1.1, 1.2],
    [1, 1.2, 1.6],
    [1, 3, 5],
    [1, 2, 3, 4, 5],
    [1, 3, 5, 7, 9],
    [1, 1.1, 1.2, 1.3, 1.4],
    [1, 1.2, 1.5, 1.8, 2]
]
fold_methods_list  = [zne.scaling.fold_global]

# PCE parameters
# extrap_methods = ["linear", "exponential"]
extrap_methods = ["exponential"]

all_results = {}

# root data folder
results_folder = "data_PCE_vs_ZNE/rand_cliffs_auto_real"
if not os.path.isdir(results_folder):
    os.makedirs(results_folder)

for total_shot_budget in shot_budgets:
    for num_qubits in qubit_list:
        num_checks     = num_qubits // 2
        extrap_checks  = range(num_checks + 1, num_qubits + 1)
        shots_per_check = total_shot_budget // num_checks

        for circuit_depth in depth_list:
            print(f"\n=== Processing n={num_qubits}, d={circuit_depth}, shots={total_shot_budget} ===")

            # check if data already exists (ignoring any _cx= suffix)
            exists, existing_path = data_exists(
                results_folder,
                num_qubits=num_qubits,
                depth=circuit_depth,
                num_circs=num_circs,
                num_samp=total_shot_budget
            )
            if exists and not overwrite:
                print(f"→ Found existing results at {existing_path}; skipping.")
                continue

            # prepare filenames (embedding the shot budget)
            file_name = (
                f"avg_errors_n={num_qubits}_d={circuit_depth}"
                f"_num_circs={num_circs}_num_samp={total_shot_budget}.csv"
            )
            plot_name = (
                f"error_plot_n={num_qubits}_d={circuit_depth}"
                f"_num_circs={num_circs}_num_samp={total_shot_budget}.png"
            )
            file_path = os.path.join(results_folder, file_name)
            plot_path = os.path.join(results_folder, plot_name)

            # --- STEP 1: experiment setup ---
            pauli_string  = "Z" * num_qubits
            only_Z_checks = True

            # --- STEP 2: generate random Clifford circuits ---
            cliff_circs = load_or_generate_random_cliffs(
                circuit_folder,
                num_qubits,
                circuit_depth,
                num_circs,
                pauli_string
            )

            # compute avg CX
            cx_counts = [circ.count_ops().get("cx", 0) for circ in cliff_circs]
            avg_cx    = float(np.mean(cx_counts))
            print(f"\n⇒ Average CX count over {num_circs} circuits: {avg_cx:.2f}")

            # --- STEP 3: compute ideal expectations ---
            ideal_expectations = [
                get_ideal_expectation(circ, pauli_string)
                for circ in cliff_circs
            ]

            # --- STEP 4: precompute PCS circuits & signs ---
            pcs_circs, signs_list = [], []
            for circ in cliff_circs:
                circ_pcs, circ_signs = [], []
                for cid in range(1, num_checks + 1):
                    sign, pcs = convert_to_PCS_circ(
                        circ, num_qubits, cid,
                        only_Z_checks=only_Z_checks,
                        barriers=True
                    )
                    circ_pcs.append(pcs)
                    circ_signs.append(sign)
                pcs_circs.append(circ_pcs)
                signs_list.append(circ_signs)

            # --- ZNE SWEEP ---
            zne_avg_errors = {}
            for zne_m in zne_methods_list:
                for sf in scale_factors_list:
                    if zne_m == "richardson" and not all(isinstance(x, int) for x in sf):
                        print(f"Skipping {zne_m} + {sf}")
                        continue

                    for fold_m in fold_methods_list:
                        print(f"\n>>> Running ZNE: method={zne_m}, scales={sf}, fold={fold_m.__name__}")
                        zne_abs = []
                        shots_each = total_shot_budget // len(sf)
                        print("shots per zne circ: ", shots_each)
                        for i, circ in enumerate(cliff_circs):
                            zne_exp = mitigate_zne(
                                circ,
                                backend,
                                pauli_string,
                                shots=shots_each,
                                method=zne_m,
                                scale_factors=sf,
                                fold_method=fold_m
                            )
                            zne_abs.append(abs(ideal_expectations[i] - zne_exp))

                        key = f"ZNE_{zne_m}_{'_'.join(map(str, sf))}_{fold_m.__name__}"
                        zne_avg_errors[key] = np.mean(zne_abs)

            print("\nZNE results:", zne_avg_errors)

            # --- PCE SWEEP ---
            pce_avg_errors = {}
            for ext in extrap_methods:
                print(f"\n>>> Running PCE: extrapolation={ext}")
                errs = []
                for i in range(len(cliff_circs)):
                    vals = []
                    for j in range(num_checks):
                        ev = ibmq_executor_pcs(
                            pcs_circs[i][j],
                            backend=backend,
                            pauli_string=pauli_string,
                            num_qubits=num_qubits,
                            shots=shots_per_check,
                            signs=signs_list[i][j]
                        )
                        vals.append(ev)
                    ex_vals, _ = extrapolate_checks(
                        num_checks,
                        extrap_checks,
                        vals,
                        method=ext
                    )
                    errs.append(abs(ideal_expectations[i] - ex_vals[-1]))
                pce_avg_errors[f"PCE_{ext}"] = np.mean(errs)

            print("\nPCE results:", pce_avg_errors)

            # --- AGGREGATE & SAVE CSV ---
            avg_errors = {**zne_avg_errors, **pce_avg_errors}
            print("saving to", results_folder)
            save_avg_errors(
                results_folder,
                file_name,
                avg_errors,
                avg_cx=avg_cx,
                overwrite=overwrite
            )